Cargamos las resoluciones en una db por cuestiones de eficiencia y performance a la hora de hacer el procesamiento posterior.
Se almacena el nombre del archivo, tipo, contenido, fecha de almacenamiento, vigencia. Por el momento la vigencia no se utiliza, pero eventualmente servirá para filtrar las normativas que ya no están vigentes.

In [1]:
!pip install PyPDF2
!pip install pdf2image
!pip install pytesseract

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.6 MB/s eta 0:00:00


In [2]:
!apt-get update
!apt-get install -y poppler-utils
!apt-get install -y tesseract-ocr tesseract-ocr-spa

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,110 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,615 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-s

In [3]:
import os
import sqlite3
from datetime import datetime
from PyPDF2 import PdfReader, errors as pdf_errors
from pdf2image import convert_from_path
import pytesseract

Ver de modularizar

In [ ]:
# Configurar rutas de Poppler y Tesseract
ruta_poppler = "/usr/bin"  # Cambia si usas otro sistema o configuración
pytesseract.pytesseract.tesseract_cmd = "/usr/bin/tesseract"

# Crear/conectar a la base de datos
db_path = "/content/normativa_dgcye.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Crear la tabla si no existe
cursor.execute('''
    CREATE TABLE IF NOT EXISTS archivos (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nombre_archivo TEXT,
        tipo_archivo TEXT,
        contenido TEXT,
        fecha_procesamiento TEXT,
        vigente BOOLEAN
    )
''')

def archivo_existe(nombre_archivo):
    cursor.execute('SELECT 1 FROM archivos WHERE nombre_archivo = ?', (nombre_archivo,))
    return cursor.fetchone() is not None  # Devuelve True si existe

# Función para insertar resoluciones en la base de datos si no existe previamente
def cargar_resoluciones(nombre_archivo, tipo_archivo, contenido):
    if archivo_existe(nombre_archivo):
        print(f"El archivo '{nombre_archivo}' ya existe en la base de datos. No se agregará.")
        return

    fecha = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    cursor.execute('''
        INSERT INTO archivos (nombre_archivo, tipo_archivo, contenido, fecha_procesamiento, vigente)
        VALUES (?, ?, ?, ?, ?)
    ''', (nombre_archivo, tipo_archivo, contenido, fecha, True))
    conn.commit()
    print(f"Archivo '{nombre_archivo}' agregado correctamente.")

# Función para extraer texto de PDFs normales
def extraer_texto_pdf(ruta_archivo):
    texto = ""
    try:
        with open(ruta_archivo, 'rb') as archivo_pdf:
            lector_pdf = PdfReader(archivo_pdf)
            for pagina in lector_pdf.pages:
                texto += pagina.extract_text() or ""
    except pdf_errors.PdfReadError as e:
        print(f"Error al leer el archivo {ruta_archivo}: {e}")
    except Exception as e:
        print(f"Ocurrió un error al procesar {ruta_archivo}: {e}")
    return texto

# Función para extraer texto usando OCR en PDFs con imágenes
def extraer_texto_ocr(ruta_archivo):
    texto = ""
    try:
        paginas_imagen = convert_from_path(ruta_archivo, poppler_path=ruta_poppler)
        for pagina_imagen in paginas_imagen:
            texto += pytesseract.image_to_string(pagina_imagen, lang='spa')
    except Exception as e:
        print(f"Error al aplicar OCR a {ruta_archivo}: {e}")
    return texto

# Función principal para procesar PDFs
def procesar_pdf(ruta_archivo):
    texto = extraer_texto_pdf(ruta_archivo)
    tipo_archivo = "PDF"
    if not texto.strip():
        print(f"Aplicando OCR al archivo {ruta_archivo}")
        texto = extraer_texto_ocr(ruta_archivo)
        tipo_archivo = "OCR"
    return texto, tipo_archivo

# Directorio con los PDFs (modifica si es necesario)
ruta_directorio = "/content/resoluciones/"
archivos_pdf = [f for f in os.listdir(ruta_directorio) if f.endswith('.pdf')]

# Procesar y cargar los PDFs en la base de datos
for archivo in archivos_pdf:
    ruta_completa = os.path.join(ruta_directorio, archivo)
    texto, tipo_archivo = procesar_pdf(ruta_completa)
    if texto:
        cargar_resoluciones(archivo, tipo_archivo, texto)

# Verificar el contenido de la base de datos
cursor.execute('SELECT * FROM archivos')
print(cursor.fetchall())

# Cerrar conexión a la base
conn.close()

Ver última fila

In [10]:
# Conectar a la base de datos SQLite
conn = sqlite3.connect("/content/normativa_dgcye.db")
cursor = conn.cursor()

cursor.execute('SELECT * FROM archivos ORDER BY id DESC LIMIT 1')
linea = cursor.fetchone()

# Verificar si se encontró la línea y mostrar el contenido
if linea:
    print(f"\nID: {linea[0]}")
    print(f"Nombre del archivo: {linea[1]}")
    print(f"Tipo de archivo: {linea[2]}")
    print(f"Contenido: {linea[3]}")
    print(f"Fecha de procesamiento: {linea[4]}")
    print(f"Vigente: {linea[5]}")
    print(f"Etiquetas: {linea[6]}")
else:
    print(f"No se encontró ninguna línea")

# Cerrar la conexión a la base de datos
conn.close()


ID: 749
Nombre del archivo: RESOLUCION_15395_1997.pdf
Tipo de archivo: PDF
Contenido: Resolución: 15395 -97 
La Plata, 5 de Diciembre de 1997  
Visto que las Secretarías de Inspección, no poseen su categorización; y  
CONSIDERANDO:  
Que en el Art. 11º de las Leyes 10579 y 10743, se establece el escalafón docente general incluyendo en el inciso a)  con los ítems VII, 
VIII y IX los cargos de Secretaria de Inspección de Primera, Segunda y Tercera categoría respectivam ente; 
Que la categorización vigente fue establecida por Resolución Nº 3576/77, que por Resolución Nº 293/8 6 se mantuvo al solo efecto de  
la liquidación de haberes;  
Que en consecuencia resulta indispensable establecer pautas conforme las normas legales vigentes;  
Que, a tal fin se han estudiado los antecedentes normativos sobre el tema;  
Que la Dirección de Gestión y Capacitación Educativa pr opone realizar la categorización conforme a la cantidad de cargos docentes 
del distrito;  
Que la Subsecretaria de Educació

## Agrego columna Etiquetas vacías por default

In [5]:
# Conectar a la base de datos SQLite
conn = sqlite3.connect("/content/normativa_dgcye.db")
cursor = conn.cursor()

try:
    cursor.execute("ALTER TABLE archivos ADD COLUMN etiquetas TEXT DEFAULT '[]'")
    conn.commit()
    print("Columna 'etiquetas' agregada exitosamente.")
except sqlite3.OperationalError as e:
    print("Error:", e)

# Cerrar la conexión a la base de datos
conn.close()

Columna 'etiquetas' agregada exitosamente.


In [6]:
conn = sqlite3.connect("/content/normativa_dgcye.db")
cursor = conn.cursor()

tabla = "archivos"
cursor.execute(f"PRAGMA table_info({tabla})")
columnas = cursor.fetchall()

# Imprimir los nombres de las columnas
print("Nombres de las columnas:")
for columna in columnas:
    print(columna[1])  # El nombre de la columna está en el índice 1

# Cerrar la conexión a la base de datos
conn.close()

Nombres de las columnas:
id
nombre_archivo
tipo_archivo
contenido
fecha_procesamiento
vigente
etiquetas
